In [1]:
import pandas as pd
from chat_bots import ChatSentimento, ChatFundamentalistas, ChatAnaliseTecnica, ChatBot
from coleta_dados import (DadosFundamentalistas, 
                          DadosNoticiasBuscadorYahoo, 
                          DadosNoticiasGoogle, 
                          CalculoWACC, 
                          VariacaoReceita, 
                          ValuationModoloGordon, 
                          OutrosAtivosNaoOperacionais,
                          PassivoTotalMenosDivida,
                          NecessidadeCapitalGiro,
                          IndicadoresFinanceiros,
                          ValuationFluxoCaixaDescontado)

from tratando_dados import tratando_dados_fundamentalistas, TratarDadosNoticias, TratandoDadosIndicadores, TratandoDadosValuation
from dotenv import load_dotenv
import ipeadatapy as ip
from datetime import datetime
import pandas as pd
import yfinance as yf
import sidrapy
import warnings
warnings.filterwarnings('ignore')
load_dotenv()


USER_AGENT environment variable not set, consider setting it to identify your requests.


True

# Fundamentalista

In [129]:
tic = 'BBDC3'
data_inicio = '2020-01-01'

dados_fundamentalistas = DadosFundamentalistas(tic, data_inicio)
trat = tratando_dados_fundamentalistas(tic, data_inicio)

In [5]:
trat

[Document(metadata={'receita_liquida': 52500.0, 'resultado_bruto': 15210.0, 'lucro_liquido': 4870.0, 'tic': 'BBDC3', 'capex_tres_meses': -10.25, 'fluxo_caixa_operacional': -6280.0, 'fluxo_caixa_investimentos': 8500.0, 'fluxo_caixa_financiamento': -4050.0, 'aumento_reducao_caixa_equivalentes': -1930.0, 'preço_lucro': '9.32', 'preço_receita_líquida': '0.57', 'preço_fco': '-3.30', 'preço_ativo_total': '0.07', 'market_cap_empresa': 144240.0, 'dividend_yield': 0.1076999999999999, 'ativo_total': 2020000.0, 'patrimonio_liquido': 167130.0, 'valor_patrimonial_acao': '15.70', 'retorno_sobre_patrimonio_liquido_inicial': 0.088, 'retorno_sobre_ativo_inicial': 0.008, 'margem_bruta': 0.277, 'margem_liquida': 0.061, 'giro_do_ativo_inicial': '0.13', 'alavancagem_financeira': '12.07', 'passivo_patrimonio_liquido': '11.07'}, page_content='2024-09-30'),
 Document(metadata={'receita_liquida': 51500.0, 'resultado_bruto': 16940.0, 'lucro_liquido': 4120.0, 'tic': 'BBDC3', 'capex_tres_meses': -1190.0, 'fluxo_c

In [6]:
query = 'Como esta a saude financeira da bbdc3'
response = ChatFundamentalistas(query=query, dados=trat,modelo_llm="deepseek-r1-distill-llama-70b")

In [7]:
response.split('\n\n')

['<think>\nOk, preciso analisar a saúde financeira da BBDC3 com base nos indicadores fornecidos. Vou começar identificando as informações disponíveis e entendendo o que cada uma delas significa.',
 'Primeiro, vou olhar para a receita líquida. Vejo que ela tem flutuado um pouco, mas parece estar em uma tendência de crescimento gradual. Isso é positivo porque indica que a empresa está aumentando sua receita ao longo do tempo.',
 'Em seguida, o lucro líquido. Notando que o lucro líquido também tem variações, mas em alguns períodos foi bastante alto, como 7.870,00, mas em outros foi muito baixo, como 1.440,00. Isso pode indicar volatilidade nos resultados, o que pode ser preocupante.',
 'Margens brutas e líquidas estão estáveis, o que é bom. A margem bruta em torno de 27% e a líquida em torno de 5-6% indicam que a empresa mantém uma rentabilidade consistente, embora possa haver espaço para melhoria na margem líquida.',
 'Agora, olhando para os fluxos de caixa. O fluxo operacional tem sido 

# Sentimento

In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

dados_yahoo = DadosNoticiasBuscadorYahoo(acao='BBDC3', options=chrome_options)


In [3]:
dados_ya = dados_yahoo.get_news(number_paginas=1)

Sem mais páginas!


In [12]:
dados_ya

{'links': ['https://www.infomoney.com.br/onde-investir/agenda-de-dividendos-de-fevereiro-2025/',
  'https://einvestidor.estadao.com.br/mercado/ibovespa-hoje-magazine-luiza-mglu3-salta-so-4-acoes-fecham-queda/',
  'https://einvestidor.estadao.com.br/ultimas/ibovespa-hoje-magazine-luiza-mglu3-dispara-9-braskem-brkm5-sofre/',
  'https://einvestidor.estadao.com.br/investimentos/dividendos-da-semana-bradesco-taesa-e-mais-6-empresas-pagam-acionistas-2/',
  'https://www.infomoney.com.br/mercados/7-acoes-sobem-mais-de-20-e-10-caem-mais-de-5-os-destaques-do-ibovespa-em-janeiro/'],
 'titulos': ['Agenda de dividendos de fevereiro: Petrobras, Itaú e Santander pagam neste mês',
  'Ibovespa hoje: Magazine Luiza (MGLU3) salta mais de 12%; só 4 ações fecham em queda',
  'Ibovespa hoje: Magazine Luiza (MGLU3) dispara 10% e lidera ganhos; Braskem (BRKM5) sofre',
  'Dividendos da semana: Bradesco (BBDC4), Banestes (BEES4), Itaú (ITUB4) e mais 9 empresas pagam...',
  '7 ações sobem mais de 20% e 10 caem m

In [4]:
ticker = 'BBDC3'
dados_google = TratarDadosNoticias(ticker, chrome_options)

In [6]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-features=NetworkService")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-features=VizDisplayCompositor")

ticker = 'BBDC3'
news_tratado = TratarDadosNoticias(ticker, chrome_options)
dados_new = news_tratado.clean_chat_html()

Sem mais páginas!
Poucas noticias encontrada para o ticker BBDC3 no Yahoo Finance. Buscando noticias no Google


Fetching pages: 100%|##########| 1/1 [00:01<00:00,  1.24s/it]


In [8]:
dados_new.split('\n\n')

['\n New notice\n### Notícias sobre BBDC3',
 '#### Artigo 1: [Título do artigo]\n* **Data:** [Data do artigo]\n* **Conteúdo:** [ Conteúdo relevante sobre BBDC3 ]',
 '#### Artigo 2: [Título do artigo]\n* **Data:** [Data do artigo]\n* **Conteúdo:** [ Conteúdo relevante sobre BBDC3 ]',
 '#### Artigo 3: [Título do artigo]\n* **Data:** [Data do artigo]\n* **Conteúdo:** [ Conteúdo relevante sobre BBDC3 ]',
 '### Resumo',
 '* **BBDC3:** [Resumo das notícias sobre BBDC3]\n* **Notícias:**\n * [Notícia 1]\n * [Notícia 2]\n * [Notícia 3]',
 '### Detalhes',
 '* **Início da Operação:** [Data de início da operação]\n* **Inversão:** [Valor da inversão]\n* **Objetivo:** [Objetivo da empresa]\n* **Serviços:** [Serviços oferecidos pela empresa]\n* **Foco:** [Foco da empresa]',
 '### Notícias',
 '* [Notícia 1]: [Conteúdo da notícia]\n* [Notícia 2]: [Conteúdo da notícia]\n* [Notícia 3]: [Conteúdo da notícia]',
 '### Conclusão',
 '* **BBDC3:** [Resumo das notícias sobre BBDC3]',
 '#### Exemplo de conteúdo'

In [9]:
query = f'Qual é o sentimento sobre as notícias sobre a {ticker}'
dados_noticia = ChatSentimento(query=query, noticia=dados_new, modelo_llm="deepseek-r1-distill-llama-70b")

In [10]:
dados_noticia.split('\n\n')

['<think>\nOkay, I need to evaluate the overall sentiment of recent news about BBDC3 for an investment analysis. Let me start by going through each news article provided.',
 'First, I see several mentions of dividend payments and JCP (juros sobre capital próprio). These are positive indicators because they show that the company is generating enough profit to distribute to shareholders. High dividends can attract investors and indicate financial health.',
 "Next, there are articles about the company's strong financial results. For example, the lucro líquido (net income) was above expectations, with a 12% increase. This is a strong positive signal as it reflects good performance and stability.",
 'The expansion into international markets is another positive point. Entering new markets can lead to growth and diversification, which is great for long-term investment.',
 "The launch of a stock option plan is also positive. This can increase liquidity and attract more investors, which is bene

# Analise tecnica

In [15]:
trat_indi = TratandoDadosIndicadores(ticker=ticker, periodo= "18Y", intervalo="1mo" )


In [16]:
dados_indicadores =   trat_indi.indicadores_data_loader()


c:\Users\jefer\Documents\Ciencia-de-dados\LLMs\Agente-investimento\.venv\Lib\site-packages\finta\finta.py:828: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sig0, xpt0, af0 = True, high[0], af
c:\Users\jefer\Documents\Ciencia-de-dados\LLMs\Agente-investimento\.venv\Lib\site-packages\finta\finta.py:829: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  _sar = [low[0] - (high - low).std()]
c:\Users\jefer\Documents\Ciencia-de-dados\LLMs\Agente-investimento\.venv\Lib\site-packages\finta\finta.py:834: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be t

In [17]:
dados_indicadores

[Document(metadata={'20 period SMA': 11.89498586654663, '100 period SMA': 12.328042049407959, '200 period SMA': 8.881622960567475, '14 period RSI': 44.05831839384031, 'MACD': -0.4836052105774389, 'SIGNAL': -0.5134500794645371, 'BB_UPPER': 14.279477388183793, 'BB_MIDDLE': 11.89498586654663, 'BB_LOWER': 9.510494344909468, 'pivot': 11.23797709607337, 's1': 10.658881950171024, 's2': 10.153276061596927, 's3': 9.57418091569458, 's4': 8.995085769792233, 'r1': 11.743582984647468, 'r2': 12.322678130549814, 'r3': 12.828284019123911, 'r4': 13.333889907698008, 'VWAP.': 10.502285948554505, '14 period ADX.': 13.103376736941206, 0: 9.926870539983447}, page_content='2023-10-01'),
 Document(metadata={'20 period SMA': 11.85486011505127, '100 period SMA': 12.375607585906982, '200 period SMA': 8.924924062490463, '14 period RSI': 52.005256820318266, 'MACD': -0.36686188026805233, 'SIGNAL': -0.48413243962524016, 'BB_UPPER': 14.156178073122218, 'BB_MIDDLE': 11.85486011505127, 'BB_LOWER': 9.553542156980322, 'p

In [8]:
response = ChatAnaliseTecnica(
    query=f"Qual é a tendencia sobre a empresa {ticker}",
    dados=dados_indicadores,
    modelo_llm="deepseek-r1-distill-llama-70b",
    stream=True
)

In [9]:
for chunk in response:
    print(chunk, end="", flush=True)

<think>
Okay, I need to analyze the trend for PETR4 using the given technical indicators. Let me go through each indicator step by step.

First, looking at the Moving Averages. The 20-period SMA is 30.14, the 100-period is 12.83, and the 200-period is 9.23. Since the 20-period is above the 100 and 200, it suggests an uptrend. Also, the 20-period SMA is increasing over time, which reinforces the uptrend.

Next, the RSI is at 64.58, which is below 70, so it's not overbought. That means there's still room for the price to go up without indicating a reversal.

The MACD is 4.62 and the Signal Line is 4.87. Since MACD is below the Signal Line, it might indicate a bearish crossover, but the values are still positive. This could mean a potential slowdown in the uptrend, but not a reversal yet.

Bollinger Bands show the upper band at 40.15, middle at 30.14, and lower at 20.12. The current price is near the middle band, which suggests moderate volatility. The bands are contracting, so volatility

In [ ]:
import yfinance as yf
ticker = "PETR4.SA"
acao = yf.Ticker(ticker,)
preco = acao.history(period="10Y", interval="1d")['Close']
preco

Date
2020-02-06 00:00:00-03:00     9.339462
2020-02-07 00:00:00-03:00     9.259444
2020-02-10 00:00:00-03:00     9.323457
2020-02-11 00:00:00-03:00     9.435479
2020-02-12 00:00:00-03:00     9.643520
                               ...    
2025-01-31 00:00:00-03:00    37.689999
2025-02-03 00:00:00-03:00    37.500000
2025-02-04 00:00:00-03:00    37.130001
2025-02-05 00:00:00-03:00    36.860001
2025-02-06 00:00:00-03:00    36.790001
Name: Close, Length: 1245, dtype: float64

In [49]:
dividendos = acao.dividends
dividendos

Date
2020-07-23 00:00:00-03:00    0.000461
2021-04-15 00:00:00-03:00    0.792834
2021-08-17 00:00:00-03:00    1.609911
2021-12-02 00:00:00-03:00    3.250487
2022-04-14 00:00:00-03:00    2.970249
2022-05-24 00:00:00-03:00    3.715490
2022-08-12 00:00:00-03:00    6.732003
2022-11-22 00:00:00-03:00    1.748708
2023-04-28 00:00:00-03:00    2.930163
2023-06-13 00:00:00-03:00    1.893576
2023-08-22 00:00:00-03:00    1.149304
2023-11-22 00:00:00-03:00    1.371248
2024-04-26 00:00:00-03:00    1.150024
2024-05-03 00:00:00-03:00    1.776022
2024-06-12 00:00:00-03:00    1.043242
2024-08-22 00:00:00-03:00    1.053200
2024-12-12 00:00:00-03:00    1.551743
2024-12-26 00:00:00-03:00    1.328207
Name: Dividends, dtype: float64

In [ ]:
acoes = pd.read_csv("C:/Users/jefer/Downloads/setor.csv")['tic'].to_list()
lista_valuation = []
for acao in acoes:
    acao_sa = acao + ".SA"
    valu = ValuationModoloGordon(acao_sa)
    try:
        dicionario = valu.preco_acao()
        lista_valuation.append(dicionario)
    except KeyError:
        print("Erro ao obter dados da ação:", acao_sa)
    #print(dicionario)
    
valuation_acoes = pd.DataFrame(lista_valuation)
valuation_acoes.sort_values(by='diferenca', ascending=False).head(40)

# Valuation

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from chat_bots import get_secret_key
from pydantic import SecretStr
from typing import List, Iterator

try:
    api_secret_groq = get_secret_key("GROQ_API_KEY")
except KeyError as exc:
    raise ValueError("API key inválida ou não definida") from exc


def ChatValuation(
    query: str,
    precos_atual_valuations: str,
    indicadores_valuation_fluxo: str,
    valuation_metodo_gordon: str,
    valuation_fluxo_caixa: str,
    api_secret: SecretStr | None = api_secret_groq,
    temperature: float = 0.5,
    modelo_llm: str = "llama-3.3-70b-versatile",
    stream: bool = False,
) -> str | Iterator[str]:
    prompt = PromptTemplate(
        input_variables=["query", "precos_atual_valuations", "indicadores_valuation_fluxo", "valuation_metodo_gordon", "valuation_fluxo_caixa"],
        template="""
        Você é um agente especializado em avaliação de empresas, responsável por determinar se uma empresa está subavaliada ou sobreavaliada.

        Para isso, você recebe duas estimativas de valuation:
        1️⃣ Valuation pelo Fluxo de Caixa Descontado (FCD)
        2️⃣ Valuation pelo Modelo de Gordon (Crescimento Perpétuo)

        Além disso, você tem acesso ao preço atual da ação, permitindo comparar as estimativas com o valor de mercado.

        Descrição dos Dados
        1. Valores da Ação 
        Voce recebe o valor atual da ação e os valores estimados pelos dois métodos de valuation. 
        
        2. Valuation pelo Modelo de Gordon
        Baseado no crescimento sustentável dos dividendos ao longo do tempo.
        Os dados incluem dividendo mediano, taxa livre de risco, beta, CAPM, taxa de crescimento sustentável e valuation estimado da ação.
        
        3. Valuation pelo Fluxo de Caixa Descontado (FCD)
        Baseado na projeção dos fluxos de caixa futuros, trazidos ao valor presente.
        Os dados incluem receita anual, EBIT, imposto, CAPEX, depreciação, fluxo de caixa e valor presente dos fluxos futuros.
        
        4. Inputs Utilizados para o Cálculo do FCD

        Indicadores utilizados na projeção, como margem EBIT, última receita, variação da receita, depreciação sobre CAPEX, relação CAPEX/receita, WACC, número de ações, dívida total, caixa disponível, outros ativos, passivos e necessidade de capital de giro.
        Tarefa do Modelo
        
        1. Comparação entre os Métodos de Valuation

        O valuation pelo modelo de Gordon está próximo do valuation pelo fluxo de caixa descontado?
        Se houver uma grande diferença entre os dois, quais fatores podem estar influenciando essa discrepância?
        💰 2. Análise da Rentabilidade e Sustentabilidade Financeira

        A empresa apresenta fluxo de caixa positivo ou negativo ao longo dos anos projetados?
        O CAPEX está elevado a ponto de comprometer o fluxo de caixa futuro?
        A geração de valor da empresa justifica o valuation calculado?
        
        3. Riscos e Inconsistências

        O crescimento da receita e do EBIT são sustentáveis ou indicam uma possível superestimação?
        O nível de endividamento é preocupante? A relação entre dívida e caixa disponível sugere riscos financeiros?
        O WACC está adequado para descontar os fluxos de caixa futuros?
        
        4. Conclusão e Recomendação

        Com base nos cálculos, a empresa está subavaliada ou sobreavaliada?
        Qual seria a recomendação para investidores com base nos dados analisados?
        Formato de Resposta Esperado
        O modelo deve fornecer uma análise estruturada e objetiva, respondendo cada questão com base nos dados fornecidos.
        A resposta deve incluir comparações numéricas, justificativas para possíveis discrepâncias entre os métodos de valuation e considerações sobre os riscos financeiros.

        REQUESTED ANALYSIS:
        Question: {query}
        Precos dos valuations e atual: {precos_atual_valuations}
        Inputs passado para estimar fluxo de caixa: {indicadores_valuation_fluxo}
        Valuation Fluxo Caixa: {valuation_fluxo_caixa}
        Valuation Metodo Gordon: {valuation_metodo_gordon}    
        
        Always response the questions in Portuguese.

      """,
    )

    model = ChatGroq(
        api_key=api_secret,
        model=modelo_llm,
        # model="llama-3.2-11b-vision-preview",
        temperature=temperature,
        stop_sequences=None,
    )

    llm_chain = prompt | model | StrOutputParser()

    if stream:
        response_stream = llm_chain.stream(input={"query": query, 
                                                  "precos_atual_valuations": precos_atual_valuations,
                                                  "indicadores_valuation_fluxo": indicadores_valuation_fluxo,
                                                  "valuation_metodo_gordon": valuation_metodo_gordon,
                                                  "valuation_fluxo_caixa": valuation_fluxo_caixa})
        return response_stream

    else:
        response_invoke = llm_chain.invoke(input={"query": query, 
                                                  "precos_atual_valuations": precos_atual_valuations,
                                                  "indicadores_valuation_fluxo": indicadores_valuation_fluxo,
                                                  "valuation_metodo_gordon": valuation_metodo_gordon,
                                                  "valuation_fluxo_caixa": valuation_fluxo_caixa})
        return response_invoke

In [3]:
ticker = 'BRAV3'
dados_valu = TratandoDadosValuation(ticker=ticker, anos_projecao=5, taxa_crescimento_perpetuidade=0.014, calculo_necessidade_capital_de_giro=False)
markdow_gordon, markdow_fluxo, markdow_preco, markdow_indicadores = dados_valu.dados_valuation()
response = ChatValuation(
    query=f"Qual é o valuation da empresa {ticker}",
    precos_atual_valuations=markdow_preco,
    indicadores_valuation_fluxo=markdow_indicadores,
    valuation_metodo_gordon=markdow_gordon,
    valuation_fluxo_caixa=markdow_fluxo,
    modelo_llm="deepseek-r1-distill-llama-70b",
    stream=True
)
for chunk in response:
    print(chunk, end="", flush=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Diferença entre valuation e cotação: nan%


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<think>
Ok, estou analisando os dados fornecidos para determinar se a empresa BRAV3 está subavaliada ou sobreavaliada. Primeiro, vou comparar os dois métodos de valuation: o Modelo de Gordon e o Fluxo de Caixa Descontado (FCD).

No Modelo de Gordon, o valuation está marcado como "nan", o que indica um problema. Isso pode ser devido a dados ausentes, como o dividendo mediano, que também está como "nan". Sem essas informações, não posso calcular o valuation por esse método. Por outro lado, o preço atual da ação é de R$ 19,93, e o valuation pelo FCD é de R$ -72.992,4. Isso já me parece estranho, pois um valuation negativo não é comum.

Agora, analisando o FCD, vejo que os fluxos de caixa projetados para os anos 2025 a 2029 são todos negativos. Isso é um sinal de alerta, pois uma empresa com fluxos de caixa consistentemente negativos pode estar enfrentando problemas financeiros graves. Além disso, o CAPEX (investimentos em ativos fixos) está muito alto, o que pode estar consumindo muito ca